In [57]:
from scipy.io import arff
import pandas as pd

data = arff.loadarff('../../data/long-method.arff')
df = pd.DataFrame(data[0])


index = df.index
number_of_rows = len(index)
print(number_of_rows)


df.head()


840


,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,...,number_static_methods,number_final_static_methods,number_final_not_static_methods,number_not_final_static_methods,number_not_final_not_static_methods,number_standard_design_methods,number_constructor_DefaultConstructor_methods,number_constructor_NotDefaultConstructor_methods,isStatic_method,is_long_method
0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,b'false'
1,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,b'false'
2,1.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,b'false'
3,1.0,0.0,2.0,1.0,0.0,2.0,11.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,b'false'
4,1.0,0.0,2.0,1.0,0.0,2.0,11.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,b'false'


In [58]:
# get all rows and all columns values except the last one
X = df.iloc[:, :-1].values 


In [59]:
from sklearn.preprocessing import MultiLabelBinarizer

# get classification for feature envy
Y_data = df.iloc[:, -1].values
print(Y_data[6])
# transform Y dependent data to binary equivalent
y = MultiLabelBinarizer().fit_transform(Y_data)


b'false'


In [60]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score

 
tree_clf = DecisionTreeClassifier()

tree_clf.fit(X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [61]:
# check if data has emty values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 83 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   NOP_method                                        840 non-null    float64
 1   CC_method                                         840 non-null    float64
 2   ATFD_method                                       840 non-null    float64
 3   FDP_method                                        840 non-null    float64
 4   CM_method                                         840 non-null    float64
 5   MAXNESTING_method                                 840 non-null    float64
 6   LOC_method                                        840 non-null    float64
 7   CYCLO_method                                      840 non-null    float64
 8   NMCS_method                                       840 non-null    float64
 9   NOLV_method          

In [62]:

from sklearn.impute import SimpleImputer
# copy data frame
X_copy = df.iloc[:, :-1].copy()
# https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html
#  fix missing data in a column by adding the median value
imputer = SimpleImputer(strategy="median")
imputer.fit(X_copy)


new_X = imputer.transform(X_copy)
# create a new data frame
new_X_df = pd.DataFrame(new_X, columns=X_copy.columns, index=X_copy.index)

new_X_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 82 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   NOP_method                                        840 non-null    float64
 1   CC_method                                         840 non-null    float64
 2   ATFD_method                                       840 non-null    float64
 3   FDP_method                                        840 non-null    float64
 4   CM_method                                         840 non-null    float64
 5   MAXNESTING_method                                 840 non-null    float64
 6   LOC_method                                        840 non-null    float64
 7   CYCLO_method                                      840 non-null    float64
 8   NMCS_method                                       840 non-null    float64
 9   NOLV_method          

In [63]:
from sklearn.model_selection import train_test_split

# //only 15 % of data set for test
# 85% for training
# find what is the meaning of the random state
X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size=0.15, random_state=42)


In [64]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import SCORERS as sco

# print(sorted(sco.keys()))

tree_clf = DecisionTreeClassifier()
# //train model
cross_val_score(tree_clf, X_train, y_train, cv=10, scoring="accuracy")

array([0.66666667, 0.68055556, 0.72222222, 0.75      , 0.63380282,
       0.66197183, 0.69014085, 0.71830986, 0.6056338 , 0.63380282])

In [65]:
# //train model
cross_val_score(tree_clf, X_train, y_train, cv=10, scoring="f1_micro")

array([0.79820628, 0.78581979, 0.81341108, 0.83599419, 0.73391813,
       0.77352941, 0.788774  , 0.80945347, 0.73444613, 0.76030534])

In [66]:
from sklearn.model_selection import GridSearchCV
import numpy as np

depths = np.arange(1, 21)

num_leafs = [1, 5, 10, 20, 50, 100]

param_grid = { 'criterion':['gini','entropy'],'max_depth': depths, 'min_samples_leaf': num_leafs}

new_tree_clf = DecisionTreeClassifier()

grid_search = GridSearchCV(new_tree_clf, param_grid, cv=10, scoring="accuracy", return_train_score=True)

grid_search.fit(X_train, y_train)



GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20]),
                         'min_samples_leaf': [1, 5, 10, 20, 50, 100]},
             return_train_score=True, scoring='accuracy')

In [67]:
grid_search.best_estimator_   

DecisionTreeClassifier(max_depth=1)

In [68]:
from sklearn.metrics import accuracy_score


 

best_model = grid_search.best_estimator_

tree_clf.fit(X_train, y_train)

 

# Test accuracy

print("The training accuracy is: ")

print("DT default: " + str(accuracy_score(y_train, tree_clf.predict(X_train))))

print("DT optimized: " + str(accuracy_score(y_train, best_model.predict(X_train))))

 

# Test accuracy

print("\nThe test accuracy is: ")

print("DT default: " + str(accuracy_score(y_test, tree_clf.predict(X_test))))

print("DT optimized: " + str(accuracy_score(y_test, best_model.predict(X_test))))

The training accuracy is: 
DT default: 0.7394957983193278
DT optimized: 0.8333333333333334

The test accuracy is: 
DT default: 0.6666666666666666
DT optimized: 0.8333333333333334


In [69]:
from sklearn.metrics import f1_score


# Test f1 score

print("The training f1 score is: ")

print("DT default: " + str(f1_score(y_train, tree_clf.predict(X_train), average='micro')))

print("DT optimized: " + str(f1_score(y_train, best_model.predict(X_train), average='micro')))

 

# Test accuracy

print("\nThe test f1 score is: ")

print("DT default: " + str(f1_score(y_test, tree_clf.predict(X_test), average='micro')))

print("DT optimized: " + str(f1_score(y_test, best_model.predict(X_test), average='micro')))

The training f1 score is: 
DT default: 0.8958566629339305
DT optimized: 0.8813559322033899

The test f1 score is: 
DT default: 0.7664172901080631
DT optimized: 0.8813559322033899
